<a href="https://colab.research.google.com/github/emilywingler/ED2-VariantesDNA/blob/main/Secao07_An%C3%A1lise_de_Sentimentos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importando as bibliotecas

In [ ]:
!pip install tensorflow --quiet

In [ ]:
!pip install translate --quiet

In [ ]:
# Para o LSTM
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, SpatialDropout1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
# Para o Vader
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
mas = SentimentIntensityAnalyzer()
from textblob import TextBlob
from translate import Translator

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [81]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [ ]:
Tweets = pd.read_csv('datasets//Tweets.csv')
Tweets.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


# Análise Exploratória

In [ ]:
Tweets.groupby('airline_sentiment').count()

,tweet_id,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
airline_sentiment,,,,,,,,,,,,,,
negative,9178,9178,9178,9178,9178,32,9178,32,9178,9178,663,9178,6036,6008
neutral,3099,3099,0,1014,3099,3,3099,0,3099,3099,181,3099,2137,2128
positive,2363,2363,0,330,2363,5,2363,0,2363,2363,175,2363,1734,1684


In [ ]:
Tweets = Tweets[Tweets['airline_sentiment_confidence'] > 0.8]

In [ ]:
token = Tokenizer(num_words=500)
token.fit_on_texts(Tweets['text'].values)

In [ ]:
X = token.texts_to_sequences(Tweets['text'].values)
X = pad_sequences(X, padding="post", maxlen = 500) # Completando com 0s à direita

In [ ]:
print(X)

[[ 97  62 229 ...   0   0   0]
 [ 97  99 131 ...   0   0   0]
 [ 97   9  99 ...   0   0   0]
 ...
 [ 13 429  98 ...   0   0   0]
 [ 13  89 459 ...   0   0   0]
 [ 13   6  23 ...   0   0   0]]


# LSTM

In [ ]:
lab_enc = LabelEncoder()
y = lab_enc.fit_transform(Tweets['airline_sentiment'])
print(y)

[1 0 0 ... 0 1 0]


In [ ]:
y = to_categorical(y)
print(y)

[[0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]]


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=13)

## Camadas

In [ ]:
modelo_lstm = Sequential()

modelo_lstm.add(Embedding(input_dim=len(token.word_index), output_dim=128))
modelo_lstm.add(SpatialDropout1D(0.2))

modelo_lstm.add(LSTM(units = 196, dropout=0.2, recurrent_dropout=0.2,
                activation="tanh", recurrent_activation="sigmoid", unroll=False, use_bias=True))
modelo_lstm.add(Dense(3, activation='softmax'))

In [ ]:
modelo_lstm.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(modelo_lstm.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ spatial_dropout1d (SpatialDropout1D) │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
modelo_lstm.fit(X_train, y_train, epochs=5, batch_size=30, verbose=True, validation_data=(X_test, y_test))

Epoch 1/5
245/245 ━━━━━━━━━━━━━━━━━━━━ 483s 2s/step - accuracy: 0.6974 - loss: 0.8435 - val_accuracy: 0.7094 - val_loss: 0.8176
Epoch 2/5
245/245 ━━━━━━━━━━━━━━━━━━━━ 480s 2s/step - accuracy: 0.7098 - loss: 0.8090 - val_accuracy: 0.7094 - val_loss: 0.8052
Epoch 3/5
245/245 ━━━━━━━━━━━━━━━━━━━━ 527s 2s/step - accuracy: 0.7090 - loss: 0.8080 - val_accuracy: 0.7094 - val_loss: 0.8080
Epoch 4/5
245/245 ━━━━━━━━━━━━━━━━━━━━ 500s 2s/step - accuracy: 0.7047 - loss: 0.8141 - val_accuracy: 0.7094 - val_loss: 0.8048
Epoch 5/5
245/245 ━━━━━━━━━━━━━━━━━━━━ 518s 2s/step - accuracy: 0.7040 - loss: 0.8158 - val_accuracy: 0.7094 - val_loss: 0.8179


In [ ]:
loss, accuracy = modelo_lstm.evaluate(X_test, y_test)
print("Loss: ", loss)
print("Acurácia: ", accuracy)

99/99 ━━━━━━━━━━━━━━━━━━━━ 26s 266ms/step - accuracy: 0.7065 - loss: 0.8241
Loss:  0.8178983926773071
Acurácia:  0.7093690037727356


In [ ]:
prev = modelo_lstm.predict(X_test)
print(prev)

99/99 ━━━━━━━━━━━━━━━━━━━━ 27s 267ms/step
[[0.77295566 0.12867647 0.09836785]
 [0.77295566 0.12867647 0.09836785]
 [0.77295566 0.12867647 0.09836785]
 ...
 [0.77295566 0.12867647 0.09836785]
 [0.7729557  0.12867649 0.09836783]
 [0.7729557  0.12867649 0.09836783]]


# Vader

Negativo

In [ ]:
a = 'I really HATE this food!!!!!'
x = mas.polarity_scores(a)
print(x)

{'neg': 0.663, 'neu': 0.337, 'pos': 0.0, 'compound': -0.7842}


Neutro

In [ ]:
b = 'This is a chair.'
x = mas.polarity_scores(b)
print(x)

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}


Positivo

In [ ]:
c = 'I love this food!!! :)'
x = mas.polarity_scores(c)
print(x)

{'neg': 0.0, 'neu': 0.198, 'pos': 0.802, 'compound': 0.8433}


In [ ]:
print(type(x))

<class 'dict'>


In [ ]:
print(x['compound'])

0.8433


Para analisar sentimentos em outra língua sem ser inglês.

In [ ]:
!pip install deep_translator --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.7 MB/s eta 0:00:00


In [ ]:
from deep_translator import GoogleTranslator
traducao = GoogleTranslator(source='auto', target='en').translate("Eu amo cantar e dançar")
print(traducao)

I love singing and dancing


In [ ]:
x = mas.polarity_scores(traducao)
print(x)

{'neg': 0.0, 'neu': 0.417, 'pos': 0.583, 'compound': 0.6369}


# Comparação entre os modelos

In [ ]:
Tweets2 = pd.read_csv('datasets//Tweets2.csv')
Tweets2.head()

,id,local,sentiment,text
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [ ]:
Tweets2.head()

,id,local,sentiment,text
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [ ]:
Tweets2.groupby('sentiment').count()

,id,local,text
sentiment,,,
Irrelevant,12990,12990,12875
Negative,22542,22542,22358
Neutral,18318,18318,18108
Positive,20832,20832,20655


In [ ]:
Tweets2.loc[Tweets2['sentiment'] == 'Irrelevant', 'sentiment'] = 'Neutral'

In [ ]:
Tweets2.groupby('sentiment').count()

,id,local,text
sentiment,,,
Negative,22542,22542,22358
Neutral,31308,31308,30983
Positive,20832,20832,20655


In [ ]:
Tweets2 = Tweets2.dropna(subset=['text'])
Tweets.reset_index(drop=True, inplace=True)

## Supervisionado

In [ ]:
token = Tokenizer(num_words=500)
token.fit_on_texts(Tweets2['text'].values)

In [ ]:
X = token.texts_to_sequences(Tweets2['text'].values)
X = pad_sequences(X, padding="post", maxlen = 500)

In [ ]:
lab_enc = LabelEncoder()
y = lab_enc.fit_transform(Tweets2['sentiment'])
print(y)

[2 2 2 ... 2 2 2]


In [ ]:
y = to_categorical(y)
print(y)

[[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=13)

In [ ]:
modelo = Sequential()

modelo.add(Embedding(input_dim=len(token.word_index), output_dim=128))
modelo.add(SpatialDropout1D(0.2))

modelo.add(LSTM(units = 196, dropout=0.2, recurrent_dropout=0.2,
                activation="tanh", recurrent_activation="sigmoid", unroll=False, use_bias=True))
modelo.add(Dense(3, activation='softmax'))

In [ ]:
modelo.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(modelo.summary())

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ spatial_dropout1d_2                  │ ?                           │               0 │
│ (SpatialDropout1D)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
modelo.fit(X_train, y_train, epochs=5, batch_size=500, verbose=True, validation_data=(X_test, y_test))

Epoch 1/5
104/104 ━━━━━━━━━━━━━━━━━━━━ 194s 2s/step - accuracy: 0.4069 - loss: 1.0857 - val_accuracy: 0.4229 - val_loss: 1.0827
Epoch 2/5
104/104 ━━━━━━━━━━━━━━━━━━━━ 202s 2s/step - accuracy: 0.4161 - loss: 1.0836 - val_accuracy: 0.4229 - val_loss: 1.0812
Epoch 3/5
104/104 ━━━━━━━━━━━━━━━━━━━━ 198s 2s/step - accuracy: 0.4165 - loss: 1.0840 - val_accuracy: 0.4229 - val_loss: 1.0809
Epoch 4/5
104/104 ━━━━━━━━━━━━━━━━━━━━ 204s 2s/step - accuracy: 0.4179 - loss: 1.0831 - val_accuracy: 0.4229 - val_loss: 1.0811
Epoch 5/5
104/104 ━━━━━━━━━━━━━━━━━━━━ 200s 2s/step - accuracy: 0.4155 - loss: 1.0842 - val_accuracy: 0.4229 - val_loss: 1.0812


In [ ]:
loss, accuracy = modelo_lstm.evaluate(X_test, y_test)
print("Loss: ", loss)
print("Acurácia: ", accuracy)

694/694 ━━━━━━━━━━━━━━━━━━━━ 186s 267ms/step - accuracy: 0.2953 - loss: 1.1023
Loss:  1.1022716760635376
Acurácia:  0.29897743463516235


In [ ]:
prev = modelo_lstm.predict(X_test)
print(prev)

694/694 ━━━━━━━━━━━━━━━━━━━━ 185s 266ms/step
[[0.33832812 0.32463703 0.33703488]
 [0.33832812 0.32463703 0.33703488]
 [0.33832812 0.32463703 0.33703488]
 ...
 [0.33832812 0.32463703 0.33703488]
 [0.33832812 0.32463703 0.33703488]
 [0.33832812 0.32463703 0.33703488]]


## Vader

In [75]:
Tweets2['vader_sentiment'] =  ''
for y in range(len(Tweets2['text'])):
    x = mas.polarity_scores(Tweets2['text'].iloc[y]) #Analisa o texto em cada linha
    del x['compound'] # Deleta o compound
    maior = max(x, key=x.get) # Pega a chave com o maior score
    Tweets2.loc[y, 'vader_sentiment'] = maior # Adiciona na nova coluna criada

In [77]:
Tweets2.groupby(['vader_sentiment']).size()

,0
vader_sentiment,
,683
neg,3660
neu,65581
pos,4755


In [78]:
Tweets2.groupby(['sentiment']).size()

,0
sentiment,
Negative,22358
Neutral,30983
Positive,20655


In [79]:
Tweets2.loc[Tweets2['vader_sentiment'] == 'neg', 'vader_sentiment'] = 'Negative'
Tweets2.loc[Tweets2['vader_sentiment'] == 'pos', 'vader_sentiment'] = 'Positive'
Tweets2.loc[Tweets2['vader_sentiment'] == 'neu', 'vader_sentiment'] = 'Neutral'

In [84]:

y_pred = Tweets2['vader_sentiment'].astype(str)
y_test = Tweets2['sentiment'].astype(str)

# Calculate the confusion matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[    0     0     0     0     0]
 [  156  1088 19840  1274     0]
 [  272  1482 27169  2060     0]
 [  255  1048 18001  1351     0]
 [    0    42   571    70     0]]


In [87]:
acuracia = accuracy_score(y_test, y_pred)
print(acuracia)

0.39647022590018616


## Resultado
* Acurácia do LSTM: 0.29897743463516235
* Acurácia do Vader: 0.39647022590018616

Conclusão: Vader performou melhor do que LSTM